In [1]:
from aeneas.executetask import ExecuteTask
from aeneas.task import Task
from aeneas.language import Language
from aeneas.syncmap import SyncMapFormat
import os
import sox

In [2]:
def audio_combine(input_file_name, output_file_name):
    # create combiner
    cbn = sox.Combiner()

    #pitch shift combined audio up 3 semitones
    #cbn.pitch(3.0)
    
    #cbn.convert(samplerate=8000, channels=2)

    # create the output file
    #cbn.build(['196tage_01_bessels_64kb.mp3', '196tage_02_bessels_64kb.mp3', '196tage_03_bessels_64kb.mp3', '196tage_04_bessels_64kb.mp3'], '196tage.mp3', 'concatenate')
    cbn.build(input_file_name, output_file_name, 'concatenate')


In [3]:
# following function is aim to split the audio file by time steps
def sox_split(input_file, output_file, start_time, end_time):

    transformer = sox.Transformer()  # create transformer
    if start_time < end_time:
        # transformer.trim(start_time=start_time, end_time=end)  # trim the audio between 0 and 15 seconds
        transformer.trim(start_time=start_time, end_time=end_time)  # trim the audio between 0 and 15 seconds

        #transformer.build('Am Quell2-ahnen1_19_freytag_64kb.mp3', 'out.mp3')  # create the output file
        transformer.build(input_file, output_file)  # create the output file

    else:
        pass

In [4]:
def get_timesteps(file):

    with open(file) as F:
        f = F.readlines()

    time_step = {'start point':[], 'end point':[]}
    for i in range(len(f)):
        #print(f[i])
        k = f[i].index('"')
        timestep = f[i][8:k]
        space = timestep.index(' ')
        start = timestep[:space]
        end = timestep[(space+1):k]
        #print('start', start)
        #print('end', end)

        time_step['start point'].append(start); time_step['end point'].append(end);

    '''
    print(time_step['start point'])
    print(time_step['end point'])
    print(len(time_step['start point']))
    print(len(time_step['end point']))
    '''
    
    return time_step

In [5]:
def write_timesteps_file(time_step, written_file):

    with open(written_file, 'w+') as f:
        txt = []
        for i in range(len(time_step['start point'])):
            txt.append(time_step['start point'][i] + ' ')
            txt.append(time_step['end point'][i] + '\n')
        f.writelines(txt)

In [6]:
def aeneas_alignemnt(text_inputs, audio_inputs, map_outputs):
    #create Task object
    '''
    config_string = u"task_language=deu|is_text_type=plain|os_task_file_format=txt"
    task = Task(config_string=config_string)
    task.audio_file_path_absolute = u"/Users/xinsun/VSCodeENV/VSCodePythonEnv/merge_der_Mantel.mp3"
    task.text_file_path_absolute = u"/Users/xinsun/VSCodeENV/VSCodePythonEnv/PS-Der_Mantel.txt"
    task.sync_map_file_path_absolute = u"/Users/xinsun/VSCodeENV/VSCodePythonEnv/syncmap.txt"
    '''
    
    config_string = u"task_language=deu|is_text_type=plain|os_task_file_format=txt|is_audio_file_detect_head_max=50||is_audio_file_detect_tail_max=30"
    #config_string = u"task_language=deu|is_text_type=plain|os_task_file_format=txt||is_audio_file_head_length=42|is_audio_file_detect_tail_min=10|is_audio_file_detect_tail_max=15"

    task = Task(config_string=config_string)
    
    task.audio_file_path_absolute =  audio_inputs
    task.text_file_path_absolute =  text_inputs
    task.sync_map_file_path_absolute = map_outputs
    
    #task.output_sync_map_file(container_root_path=None)

    #process Task
    ExecuteTask(task).execute()

    #output sync map to file
    #task.output_sync_map()
    task.output_sync_map_file(container_root_path=None)

    #print(task.sync_map)

### Following are 4 steps to get audio / text alignments on sentence level

#### Step 1. Merge several radio files into a whole radio file. / Remove the prefix or postfix of each audio file.

In [84]:
'''
# Step 1. Merge several radio files into a whole radio file.

input_file_name = ['mantel_1_gogol_64kb.mp3','mantel_2_gogol_64kb.mp3','mantel_3_gogol_64kb.mp3']

output_file_name = 'merge_der_Mantel.mp3'

audio_combine(input_file_name, output_file_name)
'''


'''
# Optional Step 1. 
# At the optional step 1, we need to remove the prefix or postfix of each audio file.

input_file_name = ['mantel_1_gogol_64kb.mp3','mantel_2_gogol_64kb.mp3','mantel_3_gogol_64kb.mp3']

output_file_name = 'merge_der_Mantel.mp3'

for i in range(len(input_file_name)):

    sox_split(input_file_name[i], output_file_name[i], start_time, end_time)

'''


"\n# Optional Step 1. \n# At the optional step 1, we need to remove the prefix or postfix of each audio file.\n\ninput_file_name = ['mantel_1_gogol_64kb.mp3','mantel_2_gogol_64kb.mp3','mantel_3_gogol_64kb.mp3']\n\noutput_file_name = 'merge_der_Mantel.mp3'\n\nfor i in range(len(input_file_name)):\n\n    sox_split(input_file_name[i], output_file_name[i], start_time, end_time)\n\n"

#### Step 2. use aeneas to get the corresponding map_alignment txt.

In [89]:
# After step 1, we then need to use aeneas to get the corresponding map_alignment txt.
# Step 2. use aeneas to get the corresponding map_alignment txt.

path_book = '/Users/xinsun/Desktop/111.nibelungenlied_ak_librivox_64kb_mp3/'

dirname_audiofiles = os.listdir(path_book)
#print(dirname_audiofiles)

In [90]:
text_all = path_book + 'transcripts/for_alignments/'
audio_all = path_book + 'audios/'
alignment_all = path_book + 'alignments/'


text_inputs, audio_inputs = [], []

[text_inputs.append(i) for i in os.listdir(text_all) if i != '.DS_Store']

[audio_inputs.append(i) for i in os.listdir(audio_all) if i != '.DS_Store']

map_outputs = alignment_all + 'maps/'

#print(len(text_inputs))
#print(len(audio_inputs))
#print(audio_inputs)
#print(text_inputs)


for i in text_inputs:
    text_input = text_all + i
    audio_input = audio_all + i.replace('.txt', '.mp3')
    map_output = map_outputs + 'map-' + i

    #print(text_input)
    #print(audio_input)
    #print(map_output)
    print('{} is completed!'.format(i.replace('.txt', '')))

    aeneas_alignemnt(text_input, audio_input, map_output)


00014-nibelungenlied is completed!
00019-nibelungenlied is completed!
00009-nibelungenlied is completed!
00004-nibelungenlied is completed!
00039-nibelungenlied is completed!
00034-nibelungenlied is completed!
00024-nibelungenlied is completed!
00029-nibelungenlied is completed!
00027-nibelungenlied is completed!
00037-nibelungenlied is completed!
00007-nibelungenlied is completed!
00017-nibelungenlied is completed!
00001-nibelungenlied is completed!
00011-nibelungenlied is completed!
00021-nibelungenlied is completed!
00031-nibelungenlied is completed!
00032-nibelungenlied is completed!
00022-nibelungenlied is completed!
00012-nibelungenlied is completed!
00002-nibelungenlied is completed!
00023-nibelungenlied is completed!
00033-nibelungenlied is completed!
00003-nibelungenlied is completed!
00013-nibelungenlied is completed!
00010-nibelungenlied is completed!
00030-nibelungenlied is completed!
00020-nibelungenlied is completed!
00036-nibelungenlied is completed!
00026-nibelungenlied

#### Step 3. Get time steps files from map_alignment from above step.

In [91]:
# Step 3. Get time steps files from map_alignment from above step.

map_file_all = []
[map_file_all.append(i) for i in os.listdir(map_outputs) if i != '.DS_Store']

for i in map_file_all:
    read_map_file = map_outputs + i
    time_step = get_timesteps(read_map_file)
    
    written_file =  alignment_all + 'time_steps/' + i.replace('map', 'time_steps')
    
    write_timesteps_file(time_step, written_file)


#### Step 4. Using time steps to to split radio and get sentence slices.

In [92]:
# Step 4. Using time steps to to split radio and get sentence slices.
print(audio_inputs)
for i in audio_inputs:
    
    split_audio_file = audio_all + i
    print(i)
    
    time_steps_file = alignment_all + 'time_steps/' + 'time_steps-' + i.replace('.mp3', '.txt') 
    #print(time_steps_file)
    
    os.mkdir(alignment_all + 'sentence_slices/' + i.replace('.mp3', ''))   # set the path for sentence_slices of each chapter / book
    
    with open(time_steps_file) as f:

        timesteps = f.readlines()
        
        for j in range(len(timesteps)):
            
            space = timesteps[j].index(' ')
            start = float(timesteps[j][:space].strip())
            #print(start)
            end = float(timesteps[j][(space + 1):].strip())
            #print(end)

            sent_slices_written_file = alignment_all + 'sentence_slices/' + i.replace('.mp3', '') + '/' + str(j+1) + '.mp3'
            #print(sent_slices_written_file)

            sox_split(split_audio_file, sent_slices_written_file, start, end)
            
            #print('spliting completed: {}'.format(j))
            
        print('{} spliting completed!'.format(i.replace('.mp3', '')))

['00013-nibelungenlied.mp3', '00003-nibelungenlied.mp3', '00033-nibelungenlied.mp3', '00023-nibelungenlied.mp3', '00020-nibelungenlied.mp3', '00030-nibelungenlied.mp3', '00010-nibelungenlied.mp3', '00006-nibelungenlied.mp3', '00016-nibelungenlied.mp3', '00026-nibelungenlied.mp3', '00036-nibelungenlied.mp3', '00038-nibelungenlied.mp3', '00035-nibelungenlied.mp3', '00025-nibelungenlied.mp3', '00028-nibelungenlied.mp3', '00015-nibelungenlied.mp3', '00018-nibelungenlied.mp3', '00008-nibelungenlied.mp3', '00005-nibelungenlied.mp3', '00024-nibelungenlied.mp3', '00029-nibelungenlied.mp3', '00039-nibelungenlied.mp3', '00034-nibelungenlied.mp3', '00009-nibelungenlied.mp3', '00004-nibelungenlied.mp3', '00014-nibelungenlied.mp3', '00019-nibelungenlied.mp3', '00017-nibelungenlied.mp3', '00007-nibelungenlied.mp3', '00037-nibelungenlied.mp3', '00027-nibelungenlied.mp3', '00031-nibelungenlied.mp3', '00021-nibelungenlied.mp3', '00011-nibelungenlied.mp3', '00001-nibelungenlied.mp3', '00002-nibelungenli

##### Use sox to remove prefix of each chapter

In [239]:
input_file = ['/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/helvia_1_seneca_64kb.mp3',
              '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/helvia_2_seneca_64kb.mp3',
             '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/helvia_3_seneca_64kb.mp3',
             '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/helvia_4_seneca_64kb.mp3']

output_file = ['/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/1.mp3',
              '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/2.mp3',
              '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/3.mp3',
              '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/4.mp3']

times = [[49,-17],
[50,-17],
[50,-17],
[52,-31]]

for i in range(len(input_file)):

    sox_split(input_file[i], output_file[i], times[i][0], sox.file_info.duration(input_file[i])+times[i][1])

##### Use sox to merge audio files

In [240]:
input_file = ['/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/1.mp3',
              '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/2.mp3',
              '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/3.mp3',
              '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/4.mp3']

output_file = '/Users/xinsun/Desktop/51.trostschrift_seneca_0911_librivox_64kb_mp3/merge.mp3'

audio_combine(input_file, output_file)

Input formats were only specified for 0 out of 4 files.The last 4 files will remain unformatted.


In [165]:
'''
map_outputs = '/Users/xinsun/Desktop/5.196_tage_auf_treibender_eisscholle_1608_librivox/alignments/maps/'
alignment_all =  '/Users/xinsun/Desktop/5.196_tage_auf_treibender_eisscholle_1608_librivox/alignments/'
map_file_all = []
[map_file_all.append(i) for i in os.listdir(map_outputs) if i != '.DS_Store']

#print(map_file_all)
split_audio_file = '/Users/xinsun/Desktop/5.196_tage_auf_treibender_eisscholle_1608_librivox/audios/merge.mp3'

time_steps_file = '/Users/xinsun/Desktop/5.196_tage_auf_treibender_eisscholle_1608_librivox/alignments/time_steps/time_steps-merge.txt'


with open(time_steps_file) as f:

    timesteps = f.readlines()

    for j in range(len(timesteps)):

        space = timesteps[j].index(' ')
        start = float(timesteps[j][:space].strip())
        #print(start)
        end = float(timesteps[j][(space + 1):].strip())
        #print(end)

        sent_slices_written_file = alignment_all + 'sentence_slices/' + str(j+1) + '.mp3'
        #print(sent_slices_written_file)

        sox_split(split_audio_file, sent_slices_written_file, start, end)

        print('spliting completed: {}'.format(j))

    print('{} spliting completed!'.format(i.strip('.mp3')))
'''

"\nmap_outputs = '/Users/xinsun/Desktop/5.196_tage_auf_treibender_eisscholle_1608_librivox/alignments/maps/'\nalignment_all =  '/Users/xinsun/Desktop/5.196_tage_auf_treibender_eisscholle_1608_librivox/alignments/'\nmap_file_all = []\n[map_file_all.append(i) for i in os.listdir(map_outputs) if i != '.DS_Store']\n\n#print(map_file_all)\nsplit_audio_file = '/Users/xinsun/Desktop/5.196_tage_auf_treibender_eisscholle_1608_librivox/audios/merge.mp3'\n\ntime_steps_file = '/Users/xinsun/Desktop/5.196_tage_auf_treibender_eisscholle_1608_librivox/alignments/time_steps/time_steps-merge.txt'\n\n\nwith open(time_steps_file) as f:\n\n    timesteps = f.readlines()\n\n    for j in range(len(timesteps)):\n\n        space = timesteps[j].index(' ')\n        start = float(timesteps[j][:space].strip())\n        #print(start)\n        end = float(timesteps[j][(space + 1):].strip())\n        #print(end)\n\n        sent_slices_written_file = alignment_all + 'sentence_slices/' + str(j+1) + '.mp3'\n        #pri

In [189]:
path = '/Users/xinsun/Desktop/aligning/'
book_inputs = []

[book_inputs.append(i) for i in os.listdir(path) if i != '.DS_Store']
print(book_inputs)
print(len(book_inputs))


for k in book_inputs:
    path_book = path + k
    print(path_book)

['34.peterchens_mondfahrt_librivox_64kb_mp3', '28.odysseus_1404_librivox', '43.lenz_rk_librivox_64kb_mp3']
3
/Users/xinsun/Desktop/aligning/34.peterchens_mondfahrt_librivox_64kb_mp3
/Users/xinsun/Desktop/aligning/28.odysseus_1404_librivox
/Users/xinsun/Desktop/aligning/43.lenz_rk_librivox_64kb_mp3


In [190]:
path = '/Users/xinsun/Desktop/aligning/'
book_inputs = []

[book_inputs.append(i) for i in os.listdir(path) if i != '.DS_Store']
print(book_inputs)

for k in book_inputs:
    path_book = path + k
    print(path_book)
    

# After step 1, we then need to use aeneas to get the corresponding map_alignment txt.
# Step 2. use aeneas to get the corresponding map_alignment txt.


    text_all = path_book + '/transcripts/for_alignments/'
    audio_all = path_book + '/audios/'
    alignment_all = path_book + '/alignments/'


    text_inputs, audio_inputs = [], []

    [text_inputs.append(i) for i in os.listdir(text_all) if i != '.DS_Store']

    [audio_inputs.append(i) for i in os.listdir(audio_all) if i != '.DS_Store']

    map_outputs = alignment_all + 'maps/'

    #print(len(text_inputs))
    #print(len(audio_inputs))
    #print(audio_inputs)
    #print(text_inputs)


    for i in text_inputs:
        text_input = text_all + i
        audio_input = audio_all + i.replace('.txt', '.mp3')
        map_output = map_outputs + 'map-' + i

        #print(text_input)
        #print(audio_input)
        #print(map_output)

        aeneas_alignemnt(text_input, audio_input, map_output)
        
        print('{} aeneas is completed!'.format(i.replace('.txt', '')))


        
        
        
    # Step 3. Get time steps files from map_alignment from above step.

    map_file_all = []
    [map_file_all.append(i) for i in os.listdir(map_outputs) if i != '.DS_Store']

    for i in map_file_all:
        read_map_file = map_outputs + i
        time_step = get_timesteps(read_map_file)

        written_file =  alignment_all + 'time_steps/' + i.replace('map', 'time_steps')

        write_timesteps_file(time_step, written_file)
        
        
        
    # Step 4. Using time steps to to split radio and get sentence slices.
    #print(audio_inputs)
    for i in audio_inputs:
    
        split_audio_file = audio_all + i
        print(i)

        time_steps_file = alignment_all + 'time_steps/' + 'time_steps-' + i.replace('.mp3', '.txt') 
        #print(time_steps_file)

        os.mkdir(alignment_all + 'sentence_slices/' + i.replace('.mp3', ''))  # set the path for sentence_slices of each chapter / book
    
        with open(time_steps_file) as f:

            timesteps = f.readlines()
        
            for j in range(len(timesteps)):
            
                space = timesteps[j].index(' ')
                start = float(timesteps[j][:space].strip())
                #print(start)
                end = float(timesteps[j][(space + 1):].strip())
                #print(end)

                sent_slices_written_file = alignment_all + 'sentence_slices/' + i.replace('.mp3', '') + '/' + str(j+1) + '.mp3'
                #print(sent_slices_written_file)

                sox_split(split_audio_file, sent_slices_written_file, start, end)

                print('spliting completed: {}'.format(j))
            
            print('{} spliting completed!'.format(i.strip('.mp3')))
            
    print('{} spliting completed!'.format(k))




['34.peterchens_mondfahrt_librivox_64kb_mp3', '28.odysseus_1404_librivox', '43.lenz_rk_librivox_64kb_mp3']
/Users/xinsun/Desktop/aligning/34.peterchens_mondfahrt_librivox_64kb_mp3
00009-Peterchens aeneas is completed!
00001-Peterchens aeneas is completed!
00014-Peterchens aeneas is completed!
00013-Peterchens aeneas is completed!
00006-Peterchens aeneas is completed!
00007-Peterchens aeneas is completed!
00012-Peterchens aeneas is completed!
00008-Peterchens aeneas is completed!
00002-Peterchens aeneas is completed!
00005-Peterchens aeneas is completed!
00010-Peterchens aeneas is completed!
00011-Peterchens aeneas is completed!
00004-Peterchens aeneas is completed!
00003-Peterchens aeneas is completed!
00013-Peterchens.mp3
spliting completed: 0
spliting completed: 1
spliting completed: 2
spliting completed: 3
spliting completed: 4
spliting completed: 5
spliting completed: 6
spliting completed: 7
spliting completed: 8
spliting completed: 9
spliting completed: 10
spliting completed: 11
s

spliting completed: 247
spliting completed: 248
spliting completed: 249
spliting completed: 250
spliting completed: 251
spliting completed: 252
spliting completed: 253
spliting completed: 254
spliting completed: 255
spliting completed: 256
spliting completed: 257
spliting completed: 258
spliting completed: 259
spliting completed: 260
spliting completed: 261
spliting completed: 262
spliting completed: 263
spliting completed: 264
spliting completed: 265
spliting completed: 266
spliting completed: 267
spliting completed: 268
spliting completed: 269
spliting completed: 270
spliting completed: 271
spliting completed: 272
spliting completed: 273
spliting completed: 274
spliting completed: 275
spliting completed: 276
spliting completed: 277
spliting completed: 278
spliting completed: 279
spliting completed: 280
spliting completed: 281
spliting completed: 282
spliting completed: 283
spliting completed: 284
spliting completed: 285
spliting completed: 286
spliting completed: 287
spliting complet

spliting completed: 7
spliting completed: 8
spliting completed: 9
spliting completed: 10
spliting completed: 11
spliting completed: 12
spliting completed: 13
spliting completed: 14
spliting completed: 15
spliting completed: 16
spliting completed: 17
spliting completed: 18
spliting completed: 19
spliting completed: 20
spliting completed: 21
spliting completed: 22
spliting completed: 23
spliting completed: 24
spliting completed: 25
spliting completed: 26
spliting completed: 27
spliting completed: 28
spliting completed: 29
spliting completed: 30
spliting completed: 31
spliting completed: 32
00008-Peterchens spliting completed!
00007-Peterchens.mp3
spliting completed: 0
spliting completed: 1
spliting completed: 2
spliting completed: 3
spliting completed: 4
spliting completed: 5
spliting completed: 6
spliting completed: 7
spliting completed: 8
spliting completed: 9
spliting completed: 10
spliting completed: 11
spliting completed: 12
spliting completed: 13
spliting completed: 14
spliting com

spliting completed: 54
spliting completed: 55
spliting completed: 56
spliting completed: 57
spliting completed: 58
spliting completed: 59
spliting completed: 60
spliting completed: 61
spliting completed: 62
spliting completed: 63
spliting completed: 64
spliting completed: 65
spliting completed: 66
spliting completed: 67
spliting completed: 68
00005-Peterchens spliting completed!
00010-Peterchens.mp3
spliting completed: 0
spliting completed: 1
spliting completed: 2
spliting completed: 3
spliting completed: 4
spliting completed: 5
spliting completed: 6
spliting completed: 7
spliting completed: 8
spliting completed: 9
spliting completed: 10
spliting completed: 11
spliting completed: 12
spliting completed: 13
spliting completed: 14
spliting completed: 15
spliting completed: 16
spliting completed: 17
spliting completed: 18
spliting completed: 19
spliting completed: 20
spliting completed: 21
spliting completed: 22
spliting completed: 23
spliting completed: 24
spliting completed: 25
spliting 

spliting completed: 34
spliting completed: 35
spliting completed: 36
spliting completed: 37
spliting completed: 38
spliting completed: 39
spliting completed: 40
spliting completed: 41
spliting completed: 42
spliting completed: 43
spliting completed: 44
spliting completed: 45
spliting completed: 46
spliting completed: 47
spliting completed: 48
spliting completed: 49
spliting completed: 50
spliting completed: 51
spliting completed: 52
spliting completed: 53
spliting completed: 54
spliting completed: 55
spliting completed: 56
spliting completed: 57
spliting completed: 58
spliting completed: 59
spliting completed: 60
spliting completed: 61
spliting completed: 62
spliting completed: 63
spliting completed: 64
spliting completed: 65
spliting completed: 66
spliting completed: 67
spliting completed: 68
spliting completed: 69
spliting completed: 70
spliting completed: 71
spliting completed: 72
spliting completed: 73
spliting completed: 74
spliting completed: 75
spliting completed: 76
spliting co

AudioFileUnsupportedFormatError: Audio file format not supported by ffprobe